In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import math
import matplotlib.pyplot as plt
import pickle
import functools
import scipy as sp
from sklearn.model_selection import StratifiedKFold, cross_validate
from scipy.spatial import distance
from sklearn.decomposition import PCA,qPCA
from tabulate import tabulate
from sklearn.cluster import KMeans,DMeans_
from sklearn.QuantumUtility.Utility import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('kddcup.data_10_percent_corrected',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [4]:
df1 = df.drop(columns=[1,2,3,6,11,20,21])
df1.loc[df1[41] != 'normal.', 41] = 'attack'
df1

,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
0,0,181,5450,0,0,0,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,239,486,0,0,0,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,217,2032,0,0,0,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,282,2286,0,0,0,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,203,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,291,1200,0,0,0,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


In [5]:
def systematic_sampling(df,pop_length):
    df_normal=df[df[41]!='attack']
    step = int(len(df_normal)/pop_length)
    
    
    first_index = random.choice(np.arange(step))
    indexes = np.arange(first_index, len(df), step=step)[:5000]
    
    training_set = df_normal.iloc[indexes]
    
    mean_std = training_set.drop(columns=41).describe(include='all').loc[['mean','std']]
    
    index_res = list(set(df_normal.index)-set(training_set.index))
    test = df_normal.loc[index_res]
    t = df[df[41] =='attack'].sample(39674)
    test= test.append(t)

    return training_set,test,mean_std


In [6]:
train1,test1,mean1 = systematic_sampling(df1,5000)

train2,test2,mean2 = systematic_sampling(df1,5000)

train3,test3,mean3 = systematic_sampling(df1,5000)

train4,test4,mean4 = systematic_sampling(df1,5000)

train5,test5,mean5 = systematic_sampling(df1,5000)

In [7]:
mean_std = [mean1,mean2,mean3,mean4,mean5]
test = [test1,test2,test3,test4,test5]

# ---------------------------------------------------------------------------------------------------------------

# Trimming function

In [8]:
def trimming(n_trimming, df, gamma):

    for n_tr in range(n_trimming):
        mean_array = np.array(df.drop(columns=41).describe(include='all').loc[['mean','std']].iloc[0])
        cov = np.cov(df.drop(columns=41).T)
        Icov = np.linalg.pinv(cov)
        dist_list = []
        for i in range(len(df)):
            sample_i = np.array(df.drop(columns=41).iloc[i])
            dist =distance.mahalanobis(sample_i,mean_array,Icov)**2
            dist_list.append(dist)
            #print(i)
        threshold = int(gamma*len(df))
        dist_list=np.array(dist_list)
        x = np.argpartition(dist_list,-threshold)[-threshold:]
        names =[]
    
        for i in x:
            names.append(df.iloc[i].name)
        df = df.drop(index = names)
    return df


## Trimming 1

Trimming the whole dataset and then extract 5000 samples from the training set.

In [9]:

#trimming 1 time
df1_normal = df1[df1[41]!='attack']
train = trimming(1,df1_normal,0.005)
train_trim=train.sample(5000)
#train_trim,test_trim,mean_trim = systematic_sampling(df1,5000)
mean_std_trim = train_trim.drop(columns=41).describe(include='all').loc[['mean','std']]
##TEST SET

x = list(train_trim.index)
index_res = list(set(df1_normal.index)-set(x))
test_trim = df1_normal.loc[index_res]
t = df1[df1[41] =='attack'].sample(39674)
test_trim= test_trim.append(t)
test_trim


,0,4,5,7,8,9,10,12,13,14,...,32,33,34,35,36,37,38,39,40,41
1,0,239,486,0,0,0,0,0,0,0,...,19,1.00,0.00,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,235,1337,0,0,0,0,0,0,0,...,29,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,219,1337,0,0,0,0,0,0,0,...,39,1.00,0.00,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,217,2032,0,0,0,0,0,0,0,...,49,1.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,normal.
5,0,217,2032,0,0,0,0,0,0,0,...,59,1.00,0.00,0.02,0.0,0.0,0.0,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299144,0,1032,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
95428,0,1032,0,0,0,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,attack
388301,0,0,0,0,0,0,0,0,0,0,...,15,0.06,0.06,0.00,0.0,1.0,1.0,0.0,0.0,attack
352935,0,0,0,0,0,0,0,0,0,0,...,2,0.01,0.07,0.00,0.0,1.0,1.0,0.0,0.0,attack


## Trimming 2

Trimming the 5000 connections of the training set

In [9]:
train1 = trimming(1,train1,0.005)
train2 = trimming(1,train2,0.005)
train3 = trimming(1,train3,0.005)
train4 = trimming(1,train4,0.005)
train5 = trimming(1,train5,0.005)

In [10]:
trains = [train1,train2,train3,train4,train5]

# ---------------------------------------------------------------------------------------------------------------

## Normalize training set and test set


In [11]:
scaler = StandardScaler()
for train_ in trains:
    for (columnName, columnData) in train_.drop(columns=41).iteritems():
        train_[columnName] = scaler.fit_transform(train_.drop(columns=41)[[columnName]])

In [12]:
#Remove mean and std from test set (Normalize test set)
tests =[]
for en, test_ in enumerate(test):
    test_ = (test_.drop(columns=41).sub(mean_std[en].loc['mean'],axis = 'columns')).div(mean_std[en].loc['std'],axis = 'columns').fillna(0)
    tests.append(test_)


# Fit PCA 

In [58]:
#Fit PCAs
pca30 = PCA(svd_solver="full").fit(trains[0].drop(columns=41))

pca40 = PCA(svd_solver="full").fit(trains[1].drop(columns=41))

pca50 = PCA(svd_solver="full").fit(trains[2].drop(columns=41))

pca60 = PCA(svd_solver="full").fit(trains[3].drop(columns=41))

pca70 = PCA(svd_solver="full").fit(trains[4].drop(columns=41))

PCAs=[pca30,pca40,pca50,pca60,pca70]


In [13]:
#Fit PCAs
pca30 = qPCA(svd_solver="full").fit(trains[0].drop(columns=41),classic_ret_variance_components=0.30)

pca40 = qPCA(svd_solver="full").fit(trains[1].drop(columns=41),classic_ret_variance_components=0.40)

pca50 = qPCA(svd_solver="full").fit(trains[2].drop(columns=41),classic_ret_variance_components=0.50)

pca60 = qPCA(svd_solver="full").fit(trains[3].drop(columns=41),classic_ret_variance_components=0.60)

pca70 = qPCA(svd_solver="full").fit(trains[4].drop(columns=41),classic_ret_variance_components=0.70)

PCAs=[pca30,pca40,pca50,pca60,pca70]


# Empirical Distribution

In [14]:
# Compute Quantile---> 1-\alpha1
alpha = [0.01,0.02,0.04,0.06,0.08,0.10]
quantils = []
for i in alpha:
    eq =  [1,-2,i]
    quantile = 1-np.round(np.roots(eq)[1],decimals=4)
    quantils.append(quantile)
quantils

[0.995, 0.9899, 0.9798, 0.9695, 0.9592, 0.9487]

Found outlier threshold using empirical distribution of \sum_q(y_i**2)/\lambda_i and of \sum_p-r+1(y_i**2)/\lambda_i

In [15]:
dictionary_major = {}
dictionary_minor  = {}
for e,pca in enumerate(PCAs):
    
    r = len(pca.explained_variance_[pca.explained_variance_ < 0.20])
    
    p = len(pca.explained_variance_)

    out_threshold_list_major = []
    out_threshold_list_minor = []
    emp_distribution_major = []
    emp_distribution_minor = []
    
    for j in range(len(trains[e].drop(columns=41))):
        sample = np.array(trains[e].drop(columns=41).iloc[j])
        y = np.dot(sample,pca.components_.T)
        s_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])
        
        s_minor = np.sum(y[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])

        emp_distribution_major.append(s_major)
        emp_distribution_minor.append(s_minor)

    for q in quantils:
        n_major = len(emp_distribution_major)
        n_minor = len(emp_distribution_minor)

        sort_major = sorted(emp_distribution_major)
        sort_minor = sorted(emp_distribution_minor)
        out_threshold_major = sort_major[int(n_major * q)]
        out_threshold_minor = sort_minor[int(n_minor * q)]
        out_threshold_list_major.append(out_threshold_major)
        out_threshold_list_minor.append(out_threshold_minor)

    dictionary_major.update({pca.components_retained_:out_threshold_list_major})
    dictionary_minor.update({pca.components_retained_:out_threshold_list_minor})


In [187]:
#Compute Confusion Matrix using outlier threshold
# Experiment =0 -> use only major components. 
#Experiment = 1 -> use both major and minor components

experiment = 1

if experiment == 0:
    classification_res = {}
    for e,pca in enumerate(PCAs):
    
        acc_list  = []
        for threshold_major in dictionary_major[pca.components_retained_]:
            TP =[]
            FP = []
            TN = []
            FN = []

            for j in range(len(tests[e])):
                sample = np.array(tests[e].iloc[j])

                y = np.dot(sample, pca.components_.T)

                sum_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])


                if sum_major > threshold_major:
                    if test[e].iloc[j][41] == 'attack':
                        TP.append(j)
                    else:
                        FP.append(j)
                else:
                    if sum_major <= threshold_major:

                        if test[e].iloc[j][41] == 'attack':
                            FN.append(j)
                        else:
                            TN.append(j)

            accuracy = (len(TP))/(len(TP)+len(FN))
            print(accuracy)
            acc_list.append(accuracy)

        classification_res.update({pca.components_retained_:acc_list})

else:
    
    classification_res = {}
    for e,pca in enumerate(PCAs):
        r = len(pca.explained_variance_[pca.explained_variance_ < 0.20])
        p = len(pca.explained_variance_)

        acc_list  = []
        for threshold_major,threshold_minor in zip(dictionary_major[pca.components_retained_],dictionary_minor[pca.components_retained_]):
            print(threshold_major,threshold_minor)
            TP =[]
            FP = []
            TN = []
            FN = []

            for j in range(len(tests[e])):
                sample = np.array(tests[e].iloc[j])

                y = np.dot(sample, pca.components_.T)

                sum_major = np.sum(y[:pca.components_retained_]**2 / pca.explained_variance_[:pca.components_retained_])

                sum_minor = np.sum(y[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]]**2 / pca.explained_variance_[p-r:np.where(np.isclose(pca.explained_variance_,0))[0][0]])



                if sum_major > threshold_major or sum_minor > threshold_minor:
                    if test[e].iloc[j][41] == 'attack':
                        TP.append(j)
                    else:
                        FP.append(j)
                else:
                    if sum_major <= threshold_major and sum_minor <= threshold_minor:

                        if test[e].iloc[j][41] == 'attack':
                            FN.append(j)
                        else:
                            TN.append(j)

            accuracy = (len(TP))/(len(TP)+len(FN))
            print(accuracy)
            acc_list.append(accuracy)

        classification_res.update({pca.components_retained_:acc_list})



22.90747710955622 195.35870475606748
0.9800625094520341
19.99552510893396 113.59026638763022


KeyboardInterrupt: 

In [ ]:
# First experiment
if experiment = 0:

    headers = ['FAR','PCA30','PCA40','PCA50','PCA60','PCA70']
    one_perc = [classification_res[i][0] for i in components]
    two_perc = [classification_res[i][1] for i in components]
    four_perc = [classification_res[i][2] for i in components]
    six_perc = [classification_res[i][3] for i in components]
    eight_perc = [classification_res[i][4] for i in components]
    ten_perc = [classification_res[i][5] for i in components]


    one_perc.insert(0,'1%')
    two_perc.insert(0,'2%')
    four_perc.insert(0,'4%')
    six_perc.insert(0,'6%')
    eight_perc.insert(0,'8%')
    ten_perc.insert(0,'10%')

    print(tabulate([one_perc, two_perc, four_perc, six_perc, eight_perc, ten_perc], headers=headers))
else:
    #Second experiment
headers = ['FAR','PCC']
one_perc = [classification_res[i][0] for i in components]
two_perc = [classification_res[i][1] for i in components]
four_perc = [classification_res[i][2] for i in components]
six_perc = [classification_res[i][3] for i in components]
eight_perc = [classification_res[i][4] for i in components]
ten_perc = [classification_res[i][5] for i in components]

one_perc_avg = np.mean(one_perc)
one_perc_std = np.std(one_perc)

two_perc_avg = np.mean(two_perc)
two_perc_std = np.std(two_perc)

four_perc_avg = np.mean(four_perc)
four_perc_std = np.std(four_perc)

six_perc_avg = np.mean(six_perc)
six_perc_std = np.std(six_perc)


eight_perc_avg = np.mean(eight_perc)
eight_perc_std = np.std(eight_perc)

ten_perc_avg = np.mean(ten_perc)
ten_perc_std = np.std(ten_perc)

one_str = [str(one_perc_avg)+' +/- '+str(one_perc_std)]
two_str = [str(two_perc_avg)+' +/- '+str(two_perc_std)]
four_str = [str(four_perc_avg)+' +/- '+str(four_perc_std)]
six_str = [str(six_perc_avg)+' +/- '+str(six_perc_std)]
eight_str =[ str(eight_perc_avg)+' +/- '+str(eight_perc_std)]
ten_str = [str(ten_perc_avg)+' +/- '+str(ten_perc_std)]

one_str.insert(0,'1%')
two_str.insert(0,'2%')
four_str.insert(0,'4%')
six_str.insert(0,'6%')
eight_str.insert(0,'8%')
ten_str.insert(0,'10%')

print(tabulate([one_str, two_str, four_str, six_str, eight_str, ten_str], headers=headers))
    

# Quantum